In [1]:
from models import ConvRNN, SimpleConv
from torch.utils.data import DataLoader, ConcatDataset
import torch
import numpy as np
from collections import namedtuple

Hostname dhoni.ucsd.edu not defined in /conf/study_paths/study_paths.yaml. Using default paths.


Registering schoffelen2019
{'schoffelen2019': <class 'bm.studies.schoffelen2019.schoffelen2019.Schoffelen2019Recording'>}
Registering fake
{'schoffelen2019': <class 'bm.studies.schoffelen2019.schoffelen2019.Schoffelen2019Recording'>, 'fake': <class 'bm.studies.fake.FakeRecording'>}
Registering gwilliams2022
{'schoffelen2019': <class 'bm.studies.schoffelen2019.schoffelen2019.Schoffelen2019Recording'>, 'fake': <class 'bm.studies.fake.FakeRecording'>, 'gwilliams2022': <class 'bm.studies.gwilliams2022.Gwilliams2022Recording'>}
Registering hebart2023
{'schoffelen2019': <class 'bm.studies.schoffelen2019.schoffelen2019.Schoffelen2019Recording'>, 'fake': <class 'bm.studies.fake.FakeRecording'>, 'gwilliams2022': <class 'bm.studies.gwilliams2022.Gwilliams2022Recording'>, 'hebart2023': <class 'bm.studies.hebart2023.Hebart2023Recording'>}


In [2]:
# print(torch.cuda.device_count())
# for i in range(torch.cuda.device_count()):
# 	print(torch.cuda.get_device_name(i))
# 	print(torch.cuda.get_device_properties(i).total_memory / 1e9)
# 	print(torch.cuda.mem_get_info(i)[0] / 1e9)

In [3]:
in_channels = {'meg': 272}
model_chout = 768
n_subjects = 1
# args_simpleconv = {'concatenate': False, 'depth': 10, 'linear_out': False, 'complex_out': True, 'kernel_size': 3, 'dilation_growth': 2, 'dilation_period': 5, 'skip': True, 'post_skip': False, 'growth': 1.0, 'scale': None, 'rewrite': False, 'groups': 1, 'glu': 2, 'glu_context': 1, 'glu_glu': True, 'gelu': True, 'dual_path': 0, 'conv_dropout': 0.0, 'dropout_input': 0.0, 'batch_norm': True, 'relu_leakiness': 0.0, 'subject_dim': 0, 'subject_layers': True, 'embedding_scale': 1.0, 'subject_layers_dim': 'input', 'subject_layers_id': False, 'n_fft': None, 'fft_complex': True, 'merger': True, 'merger_pos_dim': 2048, 'merger_channels': 270, 'merger_dropout': 0.2, 'merger_penalty': 0.0, 'merger_per_subject': False, 'dropout': 0.0, 'dropout_rescale': True, 'initial_linear': 270, 'initial_depth': 1, 'initial_nonlin': False, 'hidden': {'meg': 320}}
args_simpleconv = {'concatenate': False, 'depth': 10, 'linear_out': False, 'complex_out': True, 'kernel_size': 3, 'dilation_growth': 2, 'dilation_period': 5, 'skip': True, 'post_skip': False, 'growth': 1.0, 'scale': None, 'rewrite': False, 'groups': 1, 'glu': 2, 'glu_context': 1, 'glu_glu': True, 'gelu': True, 'dual_path': 0, 'conv_dropout': 0.0, 'dropout_input': 0.0, 'batch_norm': True, 'relu_leakiness': 0.0, 'subject_dim': 0, 'subject_layers': False, 'embedding_scale': 1.0, 'subject_layers_dim': 'input', 'subject_layers_id': False, 'n_fft': None, 'fft_complex': True, 'merger': False, 'merger_pos_dim': 2048, 'merger_channels': 270, 'merger_dropout': 0.2, 'merger_penalty': 0.0, 'merger_per_subject': False, 'dropout': 0.0, 'dropout_rescale': True, 'initial_linear': 270, 'initial_depth': 1, 'initial_nonlin': False, 'hidden': {'meg': 320}}
# subjects = torch.tensor([18, 22, 10, 22, 23, 21, 18,  9,  0,  7, 12, 22, 19,  3,  9, 12, 13,  4,
#         20,  0, 17, 25, 21, 21,  3, 17, 17, 12,  0, 18,  6, 20, 22, 12, 11,  3,
#         15, 26,  5,  0, 21,  0, 14,  8,  7,  9, 12, 18, 17, 23,  6,  6,  9, 24,
#         14, 19,  9, 23, 17,  6, 21, 20, 21,  3,  4,  3,  8,  5,  8, 11, 15,  4,
#         14,  0,  8,  5,  1,  9,  1,  1, 25,  1,  1,  8, 10, 22,  9, 17, 23, 24,
#          7,  1, 10,  1, 26,  1, 25, 23, 21,  5,  4,  3, 15, 25, 25, 21, 18, 13,
#         15,  8,  4, 25,  4, 14,  4, 14, 19, 10, 18, 17, 25, 10, 10, 23,  1, 12,
#          5, 22, 26,  3, 10, 15,  8, 18, 21,  3, 18,  4, 21, 22, 12,  7,  1, 26,
#         21,  3, 16, 18,  3, 17, 17, 12, 18, 12,  7,  6, 16,  9, 24, 18,  2,  3,
#          8,  1, 20,  3, 21,  6,  3, 13,  6,  9,  4,  9, 12,  9, 16,  6,  5, 17,
#         14, 25, 25,  5,  2, 13,  3, 17, 25, 21, 19, 17, 18,  1, 18,  6, 24, 24,
#          0,  7, 23,  1,  6,  1, 23,  1,  7, 21, 25, 23, 12, 13,  9, 11, 26,  5,
#         18,  8,  5, 12,  3, 24,  0, 17, 18, 17, 19,  8,  5, 18,  3,  5, 16, 15,
#         24, 25, 26, 22, 11,  7,  0, 15, 14,  0, 13,  9,  1, 16, 22,  7, 10,  1,
#         24,  3,  4, 25]) # 256
subjects = torch.tensor([1]) # 1

In [4]:
model = SimpleConv(in_channels=in_channels, out_channels=model_chout,
                           n_subjects=n_subjects, **args_simpleconv)

In [6]:
model.initial_linear

Sequential(
  (0): Conv1d(272, 270, kernel_size=(1,), stride=(1,))
)

In [5]:
model.to('cuda:7');

In [6]:
model(torch.randn(1, 272, 120).to('cuda:7'), subjects = subjects.to('cuda:7'))
# model(torch.randn(1, 272, 120), subjects = subjects)
# model(torch.randn(1, 272, 120).to('cuda:0'), subjects = subjects)

initial linear: torch.Size([1, 270, 120])
torch.Size([1, 270, 120])


RuntimeError: Given groups=1, weight of size [640, 320, 1], expected input[1, 270, 120] to have 320 channels, but got 270 channels instead

In [2]:
As = torch.randn(1, 270, 120).to('cuda:0')
Bs = torch.randn(1, 270, 270).to('cuda:0')
# torch.einsum('bij,bjk->bik', As, Bs).shape
torch.einsum('bct,bcd->bdt', As, Bs).shape

torch.Size([1, 270, 120])

In [24]:
next(iter(torch.randn(1, 120, 272).to_sparse().values()))

tensor(1.0413)

In [27]:
torch.randn(1, 120, 272).to_sparse().values().shape

torch.Size([32640])

In [10]:
fake_data = np.zeros((100, 272))

In [11]:
DataLoader(fake_data, batch_size=10, shuffle=True)

In [12]:
for batch in DataLoader(fake_data, batch_size=10, shuffle=True):
    print(batch.shape)
    print(model(batch).shape)
    break

torch.Size([10, 272])


RuntimeError: values expected sparse tensor layout but got Strided

In [18]:
Datasets = namedtuple("Datasets", "train valid test")

In [19]:
Datasets

__main__.Datasets

In [13]:
batch.values()

RuntimeError: values expected sparse tensor layout but got Strided